# Make manifest files from LTK posts (or other links/products we want to get from Athena) to do training

## Step 1, Athena phase: Run query, fetch results

Initialize boto3

In [1]:
import boto3
client = boto3.client('athena')

queries = {}

### Define the query

Example query

**NOTE THAT** if you are getting Lake Formation permission problems when adding new tables in the future, you need to go into the AWS Lake Formation console, and add the "SELECT" permission of the specific *database* and *table(s)* to the IAM role of this notebook, which is currently __AWSGlueServiceSageMakerNotebookRole-test__.

In [3]:
queries['example'] = '''
select *
from "dev-rad-rs-datalake-prod-structured".ltk_service_ltks l
inner join "dev-rad-rs-datalake-prod-structured".ltk_service_ltk_products p
  on l.ltks_id = p.ltk_id
inner join "dev-rad-rs-datalake-prod-structured".rewardstyle_products_pin pp
  on p.product_id = cast(pp.products_pin_id as varchar)
where l.status = 2
and extract(year from l.date_published)=2021
and extract(month from l.date_published)=1
and extract(day from l.date_published)=1
and p.position=0
order by l.ltks_id
'''

Query 1 - Based on campaign categories

In [4]:
queries['category'] = '''
select * from (
select
ltks.product_id
, name
, description
, image_url
, retailer
, case when category in ('Fashion','Beauty') then
case when secondary_category in ('Fashion : Childrens') then 'Childrens'
when secondary_category in ('Fashion : Mens') then 'Mens' else secondary_category end
when category='Health and Wellness' then 'Wellness'
when category='Food/Beverage' then 'Food'
else category end as category
from "dev-rad-rs-datalake-prod-structured".collaboration_service_obligations ob
join "dev-rad-rs-datalake-experimental".experimental_bi_campaign_categories cat on ob.campaign_id=cat.campaign_id
join (select *
from "dev-rad-rs-datalake-prod-structured".rs_post_service_post_links
where link_type='LTK') post_ltk on post_ltk.post_id=ob.post_id
join (select ltks.ltks_id_raw as id, share_url, lp.product_id, p.name, lp.image_url, p.advertisername as retailer, p.description
from "dev-rad-rs-datalake-prod-structured".ltk_service_ltks ltks
join "dev-rad-rs-datalake-prod-structured".ltk_service_ltk_products lp on ltks.ltks_id_raw = lp.ltk_id_raw
join "dev-rad-rs-datalake-prod-structured".rewardstyle_products_pin p on cast(p.products_pin_id as varchar)=lp.product_id
where date_published>=date_parse('2019-06-01 00:00:00','%Y-%m-%d %T') and lp.image_url is not null ) ltks on post_ltk.url=ltks.share_url
where ob.obligation_type_id=9
and ob.obligation_status_id=20
and ob.date_submitted>=date_parse('2019-06-01 00:00:00','%Y-%m-%d %T')
group by 1,2,3,4,5,6)
where category in ('Mens','Food','Travel','Childrens','Wellness','Electronics', 'Toys and Games', 'Sports and Outdoors', 'Beauty : Make Up', 'Beauty : Skincare')
;
'''

Query 2 - Based on specific retailers

In [5]:
queries['retailer'] = '''
select * from (
select cast(p.products_pin_id as varchar) as product_id
, p.name
, p.description
, lp.image_url
, display_name as retailer
, case when parent_id in (72837 , 72794 , 72793 , 72586 , 72673 , 55804 ) then 'Electronics'
when parent_id in (5734 , 55872, 802 , 55847 , 55661 ) then 'Food'
when parent_id in (272 , 263, 542, 76402, 1051 , 54040 , 72599 ) then 'Mens'
when parent_id in (800, 5347, 729,55440 , 5122 , 73882, 56031 , 77824 , 77605, 56057 ) then 'Childrens'
when parent_id in (55201 , 55109) then 'Travel'
when parent_id in (293 , 414, 72744 , 55721 , 56865 , 54236 , 76076 , 75007 , 5203 , 75754 ) then 'Sports and Outdoors'
end as category
, c.clicks
, row_number() over(partition by case when parent_id in (72837 , 72794 , 72793 , 72586 , 72673 , 55804 ) then 'Electronics'
when parent_id in (5734 , 55872, 802 , 55847 , 55661 ) then 'Food'
when parent_id in (272 , 263, 542, 76402, 1051 , 54040 , 72599 ) then 'Mens'
when parent_id in (800, 5347, 729,55440 , 5122 , 73882, 56031 , 77824 , 77605, 56057 ) then 'Childrens'
when parent_id in (55201 , 55109) then 'Travel'
when parent_id in (293 , 414, 72744 , 55721 , 56865 , 54236 , 76076 , 75007 , 5203 , 75754 ) then 'Sports and Outdoors'
end order by clicks desc) as rn
from (select c.redirect_product_id
, ad.advertiser_program_name as display_name
, coalesce(ad.global_tracking_group_id,ad.advertiser_program_id) as parent_id
, count(c.id) as clicks
from "dev-rad-rs-datalake-prod-curated".clicks_rosetta_clicks c
join "dev-rad-rs-datalake-prod-curated".brand_advertiser_program ad on c.redirect_retailer=ad.advertiser_program_id
where c.event_day_timestamp>=date_parse('2020-01-01 00:00:00','%Y-%m-%d %T')
and c.browser_is_bot=false
and coalesce(ad.global_tracking_group_id,ad.advertiser_program_id) in (72837 , 72794 , 72793 , 72586 , 72673 , 55804,5734 , 55872, 802 , 55847 , 55661,272 , 263, 542, 76402, 1051 , 54040 , 72599 ,800, 5347, 729,55440 , 5122 , 73882, 56031 , 77824 , 77605, 56057, 55201 , 55109, 293 , 414, 72744 , 55721 , 56865 , 54236 , 76076 , 75007 , 5203 , 75754)
and c.api_client_id in (1141,1144,1843,1875,1907,1908,1946,1945,1948)
group by 1,2,3) c
join "dev-rad-rs-datalake-prod-structured".rewardstyle_products_pin p on c.redirect_product_id=cast(p.products_pin_id as varchar)
join "dev-rad-rs-datalake-prod-structured".ltk_service_ltk_products lp on cast(p.products_pin_id as varchar)=lp.product_id
where lp.image_url is not null
group by 1,2,3,4,5,6,7)
where rn<5000
order by 7 desc;
'''

Query 3 - Based on influencer gender / primary category

In [6]:
queries['gender'] = '''
select * from (
select p.products_pin_id as product_id
, p.name
, p.description
, lp.image_url
, category
, c.clicks
, row_number() over(partition by category order by clicks desc) as rn
from (select c.redirect_product_id
, case when pr.gender='Male' then 'Mens' else pr.primary_category end as category
, count(c.id) as clicks
from "dev-rad-rs-datalake-prod-curated".clicks_rosetta_clicks c
join "dev-rad-rs-datalake-experimental".experimental_bi_collabs_profile pr on c.account_id=pr.account_id
where c.event_day_timestamp>=date_parse('2020-01-01 00:00:00','%Y-%m-%d %T')
and c.browser_is_bot=false
and ((pr.gender='Male' and pr.primary_category='Fashion') or pr.primary_category not in ('Beauty','Fashion','Interior'))
and c.account_id!=323228
and c.api_client_id in (1141,1144,1843,1875,1907,1908,1946,1945,1948)
group by 1,2) c
join "dev-rad-rs-datalake-prod-structured".rewardstyle_products_pin p on c.redirect_product_id=cast(p.products_pin_id as varchar)
join "dev-rad-rs-datalake-prod-structured".ltk_service_ltk_products lp on cast(p.products_pin_id as varchar)=lp.product_id
where lp.image_url is not null )
where rn<=10000
order by 5 desc;
'''

### Run the query on Athena

In [7]:
query_instances = {}

for (query_name, query_string) in dict.items(queries):
    query_instances[query_name] = client.start_query_execution(
        QueryString = query_string,
        QueryExecutionContext = {
            'Database': 'dev-rad-rs-datalake-prod-curated'
        },
        ResultConfiguration={
            'OutputLocation': 's3://data-science-product-image/ltk-product-queries',
        }
    )

### Monitor the query to see when it finishes.  If it is not "Succeeded", you shall not continue with further steps.

In [11]:
all_succeeded = True
s3_results = {}

for (query_name, query_instance) in dict.items(query_instances):
    query_execution = client.get_query_execution(QueryExecutionId=query_instance['QueryExecutionId'])
    state = query_execution['QueryExecution']['Status']['State']
    print(query_name, state)
    s3output = ""
    if state == "FAILED":
        print(query_execution['QueryExecution']) #['Status']['StateChangeReason'])
        all_succeeded = False
    elif state == "SUCCEEDED":
        s3_result = query_execution['QueryExecution']['ResultConfiguration']['OutputLocation']
        s3_results[query_name] = s3_result
        print(f"{query_name} query result location is in S3 at {s3_result}")
    else:
        all_succeeded = False

if all_succeeded:
    print("All have succeeded!")
else:
    print("DO NOT CONTINUE; keep checking these states until all have succeeded.")

example SUCCEEDED
example query result location is in S3 at s3://data-science-product-image/ltk-product-queries/d8fbf7a2-8d5b-4262-9ea3-cb36517414f8.csv
category SUCCEEDED
category query result location is in S3 at s3://data-science-product-image/ltk-product-queries/5bc00657-4dd7-438e-aa8b-504b32f0ddab.csv
retailer SUCCEEDED
retailer query result location is in S3 at s3://data-science-product-image/ltk-product-queries/52c14e1e-d4c3-41cd-9234-6f37c54601b1.csv
gender SUCCEEDED
gender query result location is in S3 at s3://data-science-product-image/ltk-product-queries/9117c9cd-cf68-4f8f-9f67-5d864f941584.csv
All have succeeded!


In [12]:
s3_results

{'example': 's3://data-science-product-image/ltk-product-queries/d8fbf7a2-8d5b-4262-9ea3-cb36517414f8.csv',
 'category': 's3://data-science-product-image/ltk-product-queries/5bc00657-4dd7-438e-aa8b-504b32f0ddab.csv',
 'retailer': 's3://data-science-product-image/ltk-product-queries/52c14e1e-d4c3-41cd-9234-6f37c54601b1.csv',
 'gender': 's3://data-science-product-image/ltk-product-queries/9117c9cd-cf68-4f8f-9f67-5d864f941584.csv'}

## Step 2, Python phase: Load the results from S3 and make manifest

Download the CSV files locally

In [2]:
for (query_name, s3_result) in dict.items(s3_results):
    !aws s3 cp {s3_result} query-results-{query_name}.csv

NameError: name 's3_results' is not defined

### Reading in CSVs and Manipulating Data

In [1]:
import pandas as pd
campaign = pd.read_csv('query-results-category.csv')
retailer = pd.read_csv('query-results-retailer.csv')
gender = pd.read_csv('query-results-gender.csv')

In [2]:
campaign = campaign.rename(columns={'name': 'title','image_url':'image'})
campaign.head(2)

,product_id,title,description,image,retailer,category
0,158024666,Walmart Grocery,"Shop Walmart’s selection online anytime, anywh...",https://images.rewardstyle.com/img?v=1&width=2...,Walmart Online Grocery,Food
1,136744475,Airbrush Flawless Foundation,Free shipping and returns on Charlotte Tilbury...,https://images.rewardstyle.com/img?v=1&width=2...,NORDSTROM.com,Beauty : Make Up


In [3]:
retailer = retailer.rename(columns={'name': 'title','image_url':'image'})
retailer.head(2)

,product_id,title,description,image,retailer,category,clicks,rn
0,116357844,adidas Originals Women's Swift Run Shoes,Shop a wide selection of adidas Originals Wome...,https://images.rewardstyle.com/img?v=1&width=2...,Dick's Sporting Goods,Sports and Outdoors,22932,1
1,141077153,KitchenAid Artisan White Mixer with Hobnail Bowl,NaN,https://images.rewardstyle.com/img?v=1&width=2...,Williams-Sonoma,Food,16966,1


In [4]:
gender = gender.rename(columns={'name': 'title','image_url':'image'})
gender.head(2)

,product_id,title,description,image,category,clicks,rn
0,145161150,Esme,Esme Eyeglasses in Sesame Tortoise for Women. ...,https://images.rewardstyle.com/img?v=1&width=2...,Wellness,135,9780
1,136178852,Women's Sexy Deep V Neck Wrapped One Piece Swi...,NaN,https://images.rewardstyle.com/img?v=1&width=2...,Wellness,135,9779


In [5]:
campaign_category_counts = pd.DataFrame(data=campaign.category.value_counts()).reset_index()
campaign_category_counts.columns=['campaign_category','campaign_category_count']
campaign_category_counts

,campaign_category,campaign_category_count
0,Beauty : Make Up,7576
1,Beauty : Skincare,3114
2,Childrens,3035
3,Sports and Outdoors,1345
4,Wellness,1261
5,Food,1008
6,Toys and Games,716
7,Electronics,446
8,Mens,326
9,Travel,51


In [6]:
retailer_counts = pd.DataFrame(data=retailer.category.value_counts()).reset_index()
retailer_counts.columns=['retailer_category','retailer_category_count']
retailer_counts

,retailer_category,retailer_category_count
0,Electronics,4999
1,Sports and Outdoors,4999
2,Food,4999
3,Childrens,4999
4,Mens,2463
5,Travel,104


In [7]:
gender_counts = pd.DataFrame(data=gender.category.value_counts()).reset_index()
gender_counts.columns=['gender_category','gender_category_count']
gender_counts

,gender_category,gender_category_count
0,Food,10000
1,Mens,10000
2,Wellness,10000
3,Family,10000
4,Travel,10000


**Random Sample**

- 1000 makeup
- 2000 food
- 5000 mens
- 4000 childrens
- 1000 sports
- 1000 skincare
- 2700 electronics
- 2000 travel
- 642 toys?

In [8]:
import numpy as np
category_list = ['Mens','Food','Travel','Childrens','Electronics', 'Toys and Games', 'Sports and Outdoors', 'Beauty : Make Up', 'Beauty : Skincare']
product_count = [5750, 3000, 2588, 5000, 3163, 716, 1189, 1150, 1150]
df = pd.DataFrame(data=category_list, columns=['category'])
df['product_count']=product_count

#add counts available 
df = pd.merge(df, campaign_category_counts, how='left',left_on='category',right_on='campaign_category')
df['campaign_category_count']=np.where(df['category']=='Sports and Outdoors', 0.7*df['product_count'] , df['campaign_category_count'] )
df = pd.merge(df, retailer_counts, how='left',left_on='category',right_on='retailer_category')
df = pd.merge(df, gender_counts, how='left',left_on='category',right_on='gender_category')
df=df.fillna(0)
df=df[['category','product_count','campaign_category_count','retailer_category_count','gender_category_count']]
df['campaign_products'] = round(df[['product_count','campaign_category_count']].min(axis=1))
df['remaining_products'] = df['product_count']-df['campaign_products']
df['retailer_products'] = np.where(df['retailer_category_count']>=0.7*df['remaining_products'], df['remaining_products'], df['retailer_category_count'])
df['gender_products'] = np.where(df['gender_category_count']>=df['remaining_products']-df['retailer_products'], df['remaining_products']-df['retailer_products'], df['gender_category_count'])
df

,category,product_count,campaign_category_count,retailer_category_count,gender_category_count,campaign_products,remaining_products,retailer_products,gender_products
0,Mens,5750,326.0,2463.0,10000.0,326.0,5424.0,2463.0,2961.0
1,Food,3000,1008.0,4999.0,10000.0,1008.0,1992.0,1992.0,0.0
2,Travel,2588,51.0,104.0,10000.0,51.0,2537.0,104.0,2433.0
3,Childrens,5000,3035.0,4999.0,0.0,3035.0,1965.0,1965.0,0.0
4,Electronics,3163,446.0,4999.0,0.0,446.0,2717.0,2717.0,0.0
5,Toys and Games,716,716.0,0.0,0.0,716.0,0.0,0.0,0.0
6,Sports and Outdoors,1189,832.3,4999.0,0.0,832.0,357.0,357.0,0.0
7,Beauty : Make Up,1150,7576.0,0.0,0.0,1150.0,0.0,0.0,0.0
8,Beauty : Skincare,1150,3114.0,0.0,0.0,1150.0,0.0,0.0,0.0


In [9]:
final_df = pd.DataFrame(columns=['product_id','title','description','image','category'])

for category in category_list:
    campaign_df = campaign[campaign.category==category][['product_id','title','description','image','category']]
    retailer_df = retailer[retailer.category==category][['product_id','title','description','image','category']]
    gender_df = gender[gender.category==category][['product_id','title','description','image','category']]
    
    n_campaign_products = int(df[df.category==category]['campaign_products'])
    n_retailer_products = int(df[df.category==category]['retailer_products'])
    n_gender_products = int(df[df.category==category]['gender_products'])
    
    final_df = final_df.append(campaign_df.sample(n_campaign_products, random_state=1))
    final_df = final_df.append(retailer_df.sample(n_retailer_products, random_state=1))
    final_df = final_df.append(gender_df.sample(n_gender_products, random_state=1))



In [10]:
final_df.category.value_counts()

Mens                   5750
Childrens              5000
Electronics            3163
Food                   3000
Travel                 2588
Sports and Outdoors    1189
Beauty : Make Up       1150
Beauty : Skincare      1150
Toys and Games          716
Name: category, dtype: int64

In [11]:
len(final_df.product_id.value_counts())

20497

In [12]:
experiment_name = "new-category-20k"

Transform the data into the necessary columns to write the input manifest

In [13]:
final_df['source_image_url'] = final_df.agg(lambda x: f"https://images.rewardstyle.com/img?v=1&width=256&height=256&crop&p={x['product_id']}", axis=1)
final_df['target_image_url'] = final_df.agg(lambda x: f"s3://data-science-labeling-input/product-insight/{experiment_name}/images/{x['product_id']}.jpg", axis=1)
final_df['source-ref'] = final_df['target_image_url']

In [14]:
final_df = final_df[['product_id', 'source_image_url', 'target_image_url', 'source-ref', 'title', 'description']]

final_df.head(1)

,product_id,source_image_url,target_image_url,source-ref,title,description
7334,125245192,https://images.rewardstyle.com/img?v=1&width=2...,s3://data-science-labeling-input/product-insig...,s3://data-science-labeling-input/product-insig...,Men's PulseBOOST HD Low-Top Sneakers,Shop Men's PulseBOOST HD Low-Top Sneakers onl...


In [15]:
final_df_dict = final_df.to_dict('records')

In [16]:
final_df_dict[1]

{'product_id': 125491771,
 'source_image_url': 'https://images.rewardstyle.com/img?v=1&width=256&height=256&crop&p=125491771',
 'target_image_url': 's3://data-science-labeling-input/product-insight/new-category-20k/images/125491771.jpg',
 'source-ref': 's3://data-science-labeling-input/product-insight/new-category-20k/images/125491771.jpg',
 'title': 'Fit 1 Chinos',
 'description': 'Free shipping and returns on rag & bone Fit 1 Chinos at Nordstrom.com. <p>An extra-slim fit defines the modern look of classic four-pocket chinos crafted from stretch-woven Japanese cotton.</p>'}

In [17]:
import json 

with open('input.manifest', 'w') as file:
    for product in final_df_dict:
        file.write(json.dumps(product) + "\n")

**Upload Images to s3**

In [18]:
%cd .. 
from src.image_uploader import *
%cd -

/home/ec2-user/SageMaker/data-science-product-image
/home/ec2-user/SageMaker/data-science-product-image/notebooks


In [19]:
image_uploader = ImageUploader(final_df_dict)

In [20]:
image_uploader.upload_all()

0
100
200
300
400
500
600600

700
800
900
1000
1100
1200
Unexpected error: <class 'FileNotFoundError'>
1300
1400
1500
1600
1700
1700
1800
1900
2000
2100
2200
2300
2400
2500
26002600

2700
2800
2900
3000
3100
3200
Unexpected error: <class 'requests.exceptions.ReadTimeout'>
3300
Unexpected error: <class 'requests.exceptions.ReadTimeout'>
Unexpected error: <class 'requests.exceptions.ReadTimeout'>
3400
Unexpected error: <class 'requests.exceptions.ReadTimeout'>
3500
3600
Unexpected error: <class 'requests.exceptions.ReadTimeout'>
Unexpected error: <class 'requests.exceptions.ReadTimeout'>
cannot identify image file '142961265'Unexpected error: <class 'requests.exceptions.ReadTimeout'>

3700
3800
Unexpected error: <class 'requests.exceptions.ReadTimeout'>
Unexpected error: <class 'requests.exceptions.ReadTimeout'>
Unexpected error: <class 'requests.exceptions.ReadTimeout'>
3900
Unexpected error: <class 'requests.exceptions.ReadTimeout'>
Unexpected error: <class 'requests.exceptions.ReadTim